In [1]:
import pysdql

In [ ]:
test_all = pysdql.tpch_query([1, 3, 4, 6, 10, 14, 15, 16, 18, 19], 0, 1, verbose=False)

>> Query 1 <<
SDQL Running...
>> Optimized Query <<
lineitem_aggr = li.sum(lambda x_lineitem: ({record({"l_returnflag": x_lineitem[0].l_returnflag, "l_linestatus": x_lineitem[0].l_linestatus}): record({"sum_qty": x_lineitem[0].l_quantity, "sum_base_price": x_lineitem[0].l_extendedprice, "sum_disc_price": ((x_lineitem[0].l_extendedprice) * (((1.0) - (x_lineitem[0].l_discount)))), "sum_charge": ((((x_lineitem[0].l_extendedprice) * (((1.0) - (x_lineitem[0].l_discount))))) * (((1.0) + (x_lineitem[0].l_tax)))), "count_order": 1})}) if (x_lineitem[0].l_shipdate <= 19980902) else (None))

results = lineitem_aggr.sum(lambda x_lineitem_aggr: {x_lineitem_aggr[0].concat(x_lineitem_aggr[1]): True})

Pandas Running...
Query 1: Pass
>> Query 3 <<
SDQL Running...
>> Optimized Query <<
building = "BUILDING"
customer_part = cu.sum(lambda x_customer: ({x_customer[0].c_custkey: True}) if (x_customer[0].c_mktsegment == building) else (None))

customer_orders = ord.sum(lambda x_orders: (({x_orders[0].o_ord

Pandas Running...
Query 16: Pass
>> Query 18 <<
SDQL Running...
>> Optimized Query <<
customer_part = cu.sum(lambda x_customer: {x_customer[0].c_custkey: record({"c_custkey": x_customer[0].c_custkey, "c_name": x_customer[0].c_name})})

lineitem_aggr = li.sum(lambda x_lineitem: {x_lineitem[0].l_orderkey: x_lineitem[0].l_quantity})

lineitem_part = lineitem_aggr.sum(lambda x_lineitem_aggr: ({x_lineitem_aggr[0]: True}) if (x_lineitem_aggr[1] > 200) else (None))

customer_orders = ord.sum(lambda x_orders: (({x_orders[0].o_orderkey: record({"c_custkey": x_orders[0].o_custkey, "c_name": customer_part[x_orders[0].o_custkey].c_name, "o_orderdate": x_orders[0].o_orderdate, "o_orderkey": x_orders[0].o_orderkey, "o_totalprice": x_orders[0].o_totalprice})}) if (lineitem_part[x_orders[0].o_orderkey] != None) else (None)) if (customer_part[x_orders[0].o_custkey] != None) else (None))

lineitem_aggr = li.sum(lambda x_lineitem: ({record({"c_name": customer_orders[x_lineitem[0].l_orderkey].c_name, "c_c

In [ ]:
print(f'Whether all queries are passed: {test_all}')